In [14]:
from pathlib import Path
import pickle
import numpy as np
import pandas as pd
import os
import scipy.io as sio
from src.dataprep.make_splits_cnc import extract_data_mat, extract_data_pickle, extract_data_csv, split_df

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
path_data_dir = Path().cwd().parent.parent / "data"
print(path_data_dir)

df_labels = pd.read_csv(path_data_dir / "processed/cnc/high_level_labels_MASTER_update2020-08-06_new-jan-may-data.csv")
df_labels.head()

/home/tim/Documents/feat-store/data


,unix_date,date,cut_dir,part,file_name,tools,len_cut,no_points,signals_names,failed,failed_tools,comment
0,1540298755,2018-10-23 08:45,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_0.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,NaN,NaN,NaN
1,1540298934,2018-10-23 08:48,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_1.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,NaN,NaN,NaN
2,1540299114,2018-10-23 08:51,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_2.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,NaN,NaN,NaN
3,1540299293,2018-10-23 08:54,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_3.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,NaN,NaN,NaN
4,1540299472,2018-10-23 08:57,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_4.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,NaN,NaN,NaN


In [58]:
i = 0
cut_dir = Path(df_labels.iloc[i].cut_dir)
file_name = df_labels.iloc[i].file_name
unix_date = df_labels.iloc[i].unix_date
file_path = Path(cut_dir) / file_name

"/".join(file_path.parts[7:])


'OCT2018_KS-NPC-6FF/23OCT2018_KS-NPC-6FF/Data_KS-NPC-6FF_0.mat'

In [59]:
save_dir = Path().cwd() / "splits_scratch"
path_data_dir = Path().cwd().parent.parent / "data"

file_list = [{"file_path": path_data_dir / "raw/cnc" / "/".join(Path(cut_dir).parts[7:]) / file_name, "unix_date": unix_date, "save_dir": save_dir} for cut_dir, file_name, unix_date in zip(df_labels.cut_dir, df_labels.file_name, df_labels.unix_date)]

In [60]:
file_list

[{'file_path': PosixPath('/home/tim/Documents/feat-store/data/raw/cnc/OCT2018_KS-NPC-6FF/23OCT2018_KS-NPC-6FF/Data_KS-NPC-6FF_0.mat'),
  'unix_date': 1540298755,
  'save_dir': PosixPath('/home/tim/Documents/feat-store/notebooks/scratch/splits_scratch')},
 {'file_path': PosixPath('/home/tim/Documents/feat-store/data/raw/cnc/OCT2018_KS-NPC-6FF/23OCT2018_KS-NPC-6FF/Data_KS-NPC-6FF_1.mat'),
  'unix_date': 1540298934,
  'save_dir': PosixPath('/home/tim/Documents/feat-store/notebooks/scratch/splits_scratch')},
 {'file_path': PosixPath('/home/tim/Documents/feat-store/data/raw/cnc/OCT2018_KS-NPC-6FF/23OCT2018_KS-NPC-6FF/Data_KS-NPC-6FF_2.mat'),
  'unix_date': 1540299114,
  'save_dir': PosixPath('/home/tim/Documents/feat-store/notebooks/scratch/splits_scratch')},
 {'file_path': PosixPath('/home/tim/Documents/feat-store/data/raw/cnc/OCT2018_KS-NPC-6FF/23OCT2018_KS-NPC-6FF/Data_KS-NPC-6FF_3.mat'),
  'unix_date': 1540299293,
  'save_dir': PosixPath('/home/tim/Documents/feat-store/notebooks/scratch

In [ ]:
save_dir = Path().cwd() / "splits_scratch"

def load_cut_save_split(file_dict):
    file_path = file_dict["file_path"]
    unix_date = file_dict["unix_date"]

    if file_path.suffix == ".mat":
        df, _ = extract_data_mat(sio.loadmat(cut_dir / file_name))
    elif file_path.suffix == ".pickle":
        df, _ = extract_data_pickle(cut_dir / file_name)
    elif file_path.suffix == ".csv":
        df, _ = extract_data_csv(cut_dir / file_name)
    else:
        print("File type not recognized for file:", file_path)

    dict_cuts = split_df(df, unix_date)

    for k,v in dict_cuts.items():
        pickle_out = open(save_dir / f"{k}.pickle", "wb")
        pickle.dump(v, pickle_out)
        pickle_out.close()

In [44]:
i = 0
cut_dir = Path(df_labels.iloc[i].cut_dir)
file_name = df_labels.iloc[i].file_name
unix_date = df_labels.iloc[i].unix_date
file_path = Path(cut_dir) / file_name

if file_path.suffix == ".mat":
    df, _ = extract_data_mat(sio.loadmat(cut_dir / file_name))
elif file_path.suffix == ".pickle":
    df, _ = extract_data_pickle(cut_dir / file_name)
elif file_path.suffix == ".csv":
    df, _ = extract_data_csv(cut_dir / file_name)
else:
    print("File type not recognized for file:", file_path)

dict_cuts = split_df(df, unix_date)

for k,v in dict_cuts.items():
    pickle_out = open(Path().cwd() / "splits_scratch" / f"{k}.pickle", "wb")
    pickle.dump(v, pickle_out)
    pickle_out.close()

Starting split for:	 1540298755
Finished split for:	 1540298755


In [42]:
f = cut_dir / file_name
f.suffix


'.mat'

In [23]:
dict_cuts.keys()

dict_keys(['1540298755_22_0', '1540298755_54_0', '1540298755_54_1', '1540298755_54_2', '1540298755_54_3', '1540298755_54_4', '1540298755_54_5', '1540298755_54_6', '1540298755_54_7', '1540298755_54_8', '1540298755_54_9', '1540298755_32_0', '1540298755_32_1', '1540298755_36_0', '1540298755_36_1', '1540298755_36_2', '1540298755_36_3', '1540298755_36_4', '1540298755_36_5', '1540298755_36_6', '1540298755_36_7', '1540298755_36_8', '1540298755_57_0', '1540298755_37_0', '1540298755_37_1', '1540298755_37_2', '1540298755_37_3', '1540298755_5_0', '1540298755_15_0', '1540298755_15_1', '1540298755_15_2', '1540298755_15_3', '1540298755_15_4', '1540298755_15_5', '1540298755_15_6', '1540298755_15_7', '1540298755_15_8', '1540298755_15_9', '1540298755_15_10', '1540298755_11_0', '1540298755_23_0', '1540298755_1_0', '1540298755_1_1', '1540298755_1_2', '1540298755_21_0', '1540298755_3_0', '1540298755_13_0', '1540298755_17_0', '1540298755_17_1', '1540298755_17_2', '1540298755_17_3', '1540298755_17_4', '1540